# YANG for Dummies!

Most people are excited to hear about the new projects from NetworkToCode, but quickly discover that there are a lot of moving parts that present a steep learning curve.  Community contributions will be critical to the success and adoption of these projects.  The groundwork has already been laid for IOS and JunOS drivers, so I want to document the path to onboarding completely new drivers.  I'll demonstrate this based on David Barroso's great article titled ["YANG for dummies"](https://napalm-automation.net/yang-for-dummies/).  

- Create yang model in `data/yang/dummies/models/star-wars/napalm-star-wars.yang`
  - ```
    dummies
    └── models
       ├── napalm-star-wars
       │  └── napalm-star-wars.yang
       └── napalm-star-wars-library.json
    ```
    - Copied from [Napalm-Automation](https://napalm-automation.net/yang-for-dummies/)
  - Create the library file (minus the comment): `cat napalm-star-wars-library.json`
    ```
    {
        "ietf-yang-library:modules-state": {
            "module-set-id": "6bd894f2-9168-484e-a0bf-f3ed38d864f9",
            "module": [
                {
                    "name": "napalm-star-wars",
                    "revision": "2019-08-31",  # <- This key *must* be present, see RFC8040
                    "conformance-type": "implement"
                }
            ]
        }
    }
    ```
    [RFC8040](https://tools.ietf.org/html/rfc8040)
  - `head napalm-star-wars.yang`
    ```
    // module name
    module napalm-star-wars {

        // boilerplate
        yang-version "1";
        namespace "https://napalm-yang.readthedocs.io/yang/napalm-star-wars";

        prefix "napalm-star-wars";
        revision "2019-08-31" {  # <- This *must* match the revision above.
          description
              "initial version";
          reference "0.0.1";
        }
        ...
    ```

- TODO: actually write the parser
- TODO: translator foo
- Add driver to `./ntc_rosetta/drivers/[driver_name].py`
- TODO: tests

## The parser

This is the code that will parse the model.  Currently copy/pasta, does nothing.

REFERENCE -> the Yangify tutorials

# TODO
all of it pretty much

In [1]:
from typing import Any, Dict, Iterator, Optional, Tuple, cast

from ntc_rosetta.helpers import json_helpers as jh

from yangify import parser
from yangify.parser.text_tree import parse_indented_config
from yangify.parser import Parser, ParserData


class PersonalData(Parser):
    class Yangify(ParserData):
        path = "/napalm-star-wars:individual/personal-data"

        def extract_elements(self) -> Iterator[Tuple[str, Dict[str, Any]]]:
            for k, v in jh.query("vlan", self.native, default={}).items():
                if k == "#text":
                    continue
                yield k, cast(Dict[str, Any], v)

    def name(self) -> Optional[str]:
        v = jh.query('name."#text"', self.yy.native)
        if v is not None:
            return str(v)
        else:
            return None

    def age(self) -> int:
        return int(self.yy.key)

    def affiliation(self) -> bool:
        return not jh.query('shutdown."#standalone"', self.yy.native)


class Universe(Parser):
    class Yangify(ParserData):
        path = "/napalm-star-wars:universe/individual"
        metadata = {"key": "dev_conf", "command": "show running-config all"}

        def pre_process(self) -> None:
            self.native: Dict[str, Any] = self.root_native["dev_conf"]

    personal_data = PersonalData


class DummyParser(parser.RootParser):
    """
    DummyParser expects as native data a dictionary where the `dev_conf`
    key is reserved for the device configuration.
    """

    class Yangify(parser.ParserData):
        def init(self) -> None:
            self.root_native["dev_conf"] = parse_indented_config(
                self.root_native["dev_conf"].splitlines()
            )
            self.native["dev_conf"] = self.root_native["dev_conf"]

    universe = Universe

## The driver
We need to register the driver with rosetta so we can make use of some its utility methods.  We'll need to overload the `get_datamodel` function so we can load our custom models.

### The YANG models
ntc-rosetta is a framework, so we are able to bring-our-own-yang models to the party.  Simply tell yangson where to find the data, and we're off to the races (or the next step of debugging)!

In [7]:
from ntc_rosetta.drivers.base import Driver
from yangson.datamodel import DataModel
import pathlib

# This part looks just like the ios and junos drivers in ntc-rosetta
class DummyDriverNapalmStarWars(Driver):
    parser = DummyParser
    translator = None
    datamodel_name = "napalm_star_wars"
    
    # These are the overloads
    @classmethod
    def get_data_model(cls) -> DataModel:
        base = pathlib.Path("/ntc_dummies/data/yang")
        lib = base.joinpath("dummies/models/napalm-star-wars-library.json")
        path = [
            base.joinpath("dummies/models/napalm-star-wars"),
        ]
        return DataModel.from_file(lib, path)
    
    @classmethod
    def get_datamodel(cls) -> DataModel:
        if cls._datamodel is None:
            cls._datamodel = cls.get_data_model()
        return cls._datamodel

In [9]:
# If everything wor
dummy_driver = DummyDriverNapalmStarWars()
# Let's see if we properly loaded the DataModel from the new YANG file
print(dummy_driver.get_datamodel().ascii_tree())

+--rw napalm-star-wars:universe
   +--rw individual* [name]
      +--rw affiliation? <identityref>
      +--rw age? <age(uint16)>
      +--rw name <string>



## The dummy device
Our dummy device is going to implement configuration through YAML files.  That's absurd you say!  YAML is a horrible format!  I don't disagree, but it's suitable for pet examples and it lives up to the "dummy" device name.

In [13]:
import json
# NOTE: you will need to install a YAML library of your 
# choosing *from the jupyter console* for this to work
from ruamel.yaml import YAML

with open("data/star_wars/universe.yml") as f:
    config = f.read()

yaml = YAML()
config_data = yaml.load(config)

print(config)
print(json.dumps(config_data, indent=2))


---
universe:
  individuals:
  - affiliation: REBEL_ALLIANCE
    age: 57
    name: Obi-Wan Kenobi
  - affiliation: REBEL_ALLIANCE
    age: 19
    name: Luke Skywalker
  - affiliation: EMPIRE
    age: 42
    name: Darth Vader
  - affiliation: REBEL_ALLIANCE
    age: 896
    name: Yoda

{
  "universe": {
    "individuals": [
      {
        "affiliation": "REBEL_ALLIANCE",
        "age": 57,
        "name": "Obi-Wan Kenobi"
      },
      {
        "affiliation": "REBEL_ALLIANCE",
        "age": 19,
        "name": "Luke Skywalker"
      },
      {
        "affiliation": "EMPIRE",
        "age": 42,
        "name": "Darth Vader"
      },
      {
        "affiliation": "REBEL_ALLIANCE",
        "age": 896,
        "name": "Yoda"
      }
    ]
  }
}


In [11]:
my_universe = DummyParser(dummy_driver.get_datamodel(), native=config_data)
my_universe.process()
# parsed = dummy_driver.parse(native={"dev_conf": config})
# print(json.dumps(parsed.raw_value(), indent=4))

{}
